In [ ]:
import pandas as pd
import numpy as np
import duckdb

__Beschreibung Oliver 13.11.2025__

die Beschreibung der Schnittstelle liegt mir vor. So habe ich mein Visualisierungstool gebaut (welches aber nicht die Umsteigehinweise beinhaltet). Die Umsteigehinweise beginnen immer mit K#132#I#EVB/113#24#1.1##320191, wobei 132 das systeminterne Modul ist, welches für die Umsteigehinweise zuständig ist, Fahrzeugnummer und Haltepunkt sind selbsterklärend, die 24#1.1 ist ein Teletype + Version. Trenner ist immer ‚#‘. Das heißt, wenn ‚##‘ auftaucht, ist ein Feld leer.

In [ ]:
file_path = 'NetPeerManager.2025-11-13.1.log' #Original IVU Log Datei
file_path_new = file_path + '.reduced' #Reduzierte Datei mit Umsteigehinweisen Modul 132

## reduzieren des Logs auf Zeilen mit Umsteigehinweisen (Modul 132)

In [ ]:
with open(file_path, encoding="Latin1") as f:
    for line in f:
        s =line.split('#')
        if len(s) > 2: #kurze Einträge für die weitere Prüfung nicht berücksichtigen
            if s[1] == '132': #Auswahl der Umsteigehinweise Modul 132
                with open(file_path_new, 'a', encoding="Latin1") as f_new:
                    f_new.write(line)

## Auslesen Umsteigehinweise und Parsen

In [ ]:
ln = 0 #laufende Nummer Zeilen
arr_concat = [] #initialisieren der Liste für DataFrames

with open(file_path_new, encoding="Latin1") as f:    
    for line in f:
        ln += 1 #Zeilennummer
        ts = line[0:29]  #Timestamp extrahieren
        s = line.split('#') #Aufspalten der Zeile in Liste 
        fzg = s[3] #Fahrzeugmandant        

        #print(fzg, ln, len(s), s[0][-1:], len(s)%14, s[1])
        arr = np.array(s[6:-3]) #in NumPy Array umwandeln, nur die Datenfelder ohne Felder zu Fzg etc und ohne die letzten 3 Felder (in der Regel leer)
        arr = arr.reshape(-1,14) #Umformen in Array mit 14 Spalten    
        df = pd.DataFrame(arr) #in DataFrame umwandeln
        df['Fzg'] = fzg #Anfügen der Fahrzeugnummer als neue Spalte      
        df['timestamp'] = ts #Anfügen des Timestamps als neue Spalte
        arr_concat.append(df)

In [ ]:
result_df = pd.concat(arr_concat, ignore_index=True)
result_df.columns = ['Timestamp1', 'HaltID', 'Halt', 
                     'Mode', 'Unknown1', 'Unknown2', 'Unknown3', 
                     'LinieIntern', 'Linie', 'HaltId2', 'Halt2', 
                     'Unknown4', 'JsonText', 'Timestamp2', 
                     'Fzg', 'timestamp']

In [ ]:
result_df.shape #Größe der DataFrame anzeigen

In [ ]:
result_df.head()

## Auswertung der Anzahl nach Linien

In [ ]:
# Concatenate all DataFrames in arr_concat into a single DataFrame
result_df['Linie'].value_counts()

In [ ]:
# Concatenate all DataFrames in arr_concat into a single DataFrame
result_df['Fzg'].value_counts()